
# 1. 問題分析（要点）

* 経路数は各区間の本数を**順番に掛け合わせ**るだけ
* 倍精度・大整数は不要（都度 `mod = 1234567` を取れば安全）
* **計算量:** 時間 O(N)、空間 O(1)

---

# 2. アルゴリズム比較表（簡潔版）

| アプローチ                  | 時間計算量 | 空間計算量 | Python実装コスト | 可読性 | 標準ライブラリ     | CPython最適化   | 備考             |
| ---------------------- | ----- | ----- | ----------- | --- | ----------- | ------------ | -------------- |
| 逐次乗算＋逐次mod（採用）         | O(N)  | O(1)  | 低           | ★★★ | なし          | 乗算/剰余はC実装で高速 | オーバーフロー回避      |
| `math.prod` で一括→最後にmod | O(N)  | O(1)  | 低           | ★★☆ | `math.prod` | 中間値が巨大になる    | 非推奨（無駄に大きい中間値） |

---

# 3. Python特有の最適化ポイント

* 乗算ごとに `%= 1234567` を当てて**中間値肥大を抑止**
* ループはシンプルな `for` が最速クラス（イテレータ・内包での副作用より明確）
* 型注釈 `List[int]` で pylance 警告回避

---

# 4. HackerRankでの回答フォーマット（関数のみ）

```python
from typing import List

MOD: int = 1234567

def connectingTowns(n: int, routes: List[int]) -> int:
    """
    Return total number of paths from T1 to Tn modulo 1234567.
    
    Args:
        n: number of towns (2 <= n <= 100)
        routes: length n-1, routes[i] = number of roads from Ti to Ti+1 (1..1000)
    """
    total: int = 1
    for r in routes:
        # 中間値の肥大化を避けるため逐次でmodを取る
        total = (total * (r % MOD)) % MOD
    return total
```

---

# 5. 検証メモ（参考）

* サンプル1: `n=3, routes=[1,3]` → 1×3=3 → 3
* サンプル2: `n=4, routes=[2,2,2]` → 8 → 8

実運用では HackerRank のテンプレートが I/O を行うため、本関数のみ差し替えればOKです。


# Connecting Towns - 区間積の逐次法（剰余付き）で総経路数を求める

* [概要](#overview)
* [アルゴリズム要点 (TL;DR)](#tldr)
* [図解](#figures)
* [証明のスケッチ](#proof)
* [計算量](#complexity)
* [Python 実装（HackerRank 形式）](#impl)
* [CPython 最適化ポイント](#cpython)
* [エッジケースと検証](#edgecases)
* [FAQ](#faq)

---

<h2 id="overview">概要</h2>

**プラットフォーム/ID:** HackerRank `connecting-towns`
**問題タイトル:** Connecting Towns

**要約:**
$N$ 個の都市 $T_1, T_2, \dots, T_N$ が直線状に並び、各隣接区間 $[T_i \to T_{i+1}]$ には $r_i$ 本の道路がある。$T_1$ から $T_N$ まで都市を**番号順に**通過する経路の総数は $r_1 \cdot r_2 \cdots r_{N-1}$ に等しい。これを法 $M=1234567$ で求める。

**入出力（簡潔）:**

* 入力: 複数テストケース。各テストで $N$ と $N-1$ 個の整数 $r_i$。
* 出力: 各テストの答え（$r_1 \cdots r_{N-1} \bmod 1234567$）。

**関数シグネチャ（HackerRank 準拠・必須）:**
`connectingTowns(n: int, routes: List[int]) -> int`

---

<h2 id="tldr">アルゴリズム要点 (TL;DR)</h2>

* 経路数は **区間ごとの独立選択の積**：
  $$\text{ans} = \prod_{i=1}^{N-1} r_i$$
* オーバーフロー/巨大中間値回避のため、**逐次乗算で都度法を取る**：
  $$\text{ans} \leftarrow (\text{ans} \cdot (r_i \bmod M)) \bmod M,\quad M=1234567$$
* 計算量は $O(N)$、追加空間 $O(1)$。

---

<h2 id="figures">図解</h2>

**フローチャート（計算手順）**

```mermaid
flowchart TD
  Start[開始] --> Init[初期化: ans=1, M=1234567]
  Init --> Loop[区間ごとに処理]
  Loop --> Step["ans ← (ans * (r_i mod M)) mod M"]
  Step --> Check[i が最後か判定]
  Check -->|未了| Loop
  Check -->|完了| End[結果を返す]
```

*説明:* 各区間の本数 $r_i$ を順に掛け、毎回 $M$ で剰余を取るだけの単純なループです。

**データフロー（入力→結果）**

```mermaid
graph LR
  InN[都市数 N] --> UseN[配列長 N-1 を確認]
  InR[区間本数配列 routes] --> Fold[剰余付き乗算の畳み込み]
  UseN --> Fold
  Fold --> Out[ans を出力]
```

*説明:* $N$ に対して `routes` が $N-1$ 要素であることを前提に、畳み込み（fold）で積の剰余を計算します。

---

<h2 id="proof">証明のスケッチ</h2>

**命題:** $T_1$ から $T_N$ を番号順に通過する経路の総数は
$$
\prod_{i=1}^{N-1} r_i
$$
に等しい。

* **不変条件:** $k$ 番目の区間まで考慮した時点の集計値 $\text{ans}_k$ は
  $$\text{ans}*k=\prod*{i=1}^{k} r_i \pmod{M}$$
  を保つ。
* **基底:** $k=0$（区間未処理）では $\text{ans}_0=1$。これは空積（$1$）と一致。
* **帰納:** $k$ 段で不変条件が成立していると仮定。次の区間 $r_{k+1}$ を処理すると
  $$
  \text{ans}*{k+1} = \left(\text{ans}*k \cdot (r*{k+1}\bmod M)\right)\bmod M
  \equiv \left(\prod*{i=1}^{k} r_i\right) r_{k+1} \pmod{M}
  =\prod_{i=1}^{k+1} r_i \pmod{M}
  $$
  よって不変条件は保たれる。
* **終了性:** $N-1$ 区間は有限。ループは $N-1$ 回で終了。
* **結論:** $\text{ans}*{N-1}\equiv \prod*{i=1}^{N-1} r_i \pmod{M}$ が返るため、正しい。

---

<h2 id="complexity">計算量</h2>

* **Time:** $O(N)$（区間数に線形）
* **Space:** $O(1)$（定数個の変数のみ）

---

<h2 id="impl">Python 実装（HackerRank 形式・型注釈付き）</h2>

> **注:** HackerRank の採点では I/O はテンプレート側が担当します。ここでは **純粋関数** `connectingTowns` のみを実装します。コメントで式と実装の対応を明示しています。

```python
from __future__ import annotations
from typing import List, Final

# 法 M は問題で固定
MOD: Final[int] = 1234567

def connectingTowns(n: int, routes: List[int]) -> int:
    """
    HackerRank: connecting-towns

    目的:
        T1 -> Tn を都市番号順に通過する経路総数を M=1234567 で返す。

    数学的定義:
        ans = (∏_{i=1}^{n-1} r_i) mod M

    実装戦略:
        - 中間値肥大化を避けるため、逐次乗算のたびに mod を取る。
        - 不変条件: ループ i 完了後、ans ≡ ∏_{k=1}^{i} r_k (mod M)

    Args:
        n: 都市数 (2 <= n <= 100)
        routes: 区間本数の配列（長さ n-1, 各要素 1..1000）

    Returns:
        ans: 上記の定義に従う整数（0 <= ans < M）
    """
    # 基底: 空積は 1
    ans: int = 1
    # 逐次的に ans ← (ans * (r_i mod M)) mod M
    for r in routes:
        # KaTeX 対応メモ: ans ← (ans × (r_i mod M)) mod M
        ans = (ans * (r % MOD)) % MOD
    return ans
```

---

<h2 id="cpython">CPython 最適化ポイント</h2>

* **逐次剰余**: すべて掛けてから `% MOD` ではなく、**毎ステップで `% MOD`** を行い中間値を抑制（レジスタ局所性・GC 回避にも有利）。
* **整数演算は C 実装**: Python の `int` 乗算・剰余は内部で最適化済み。不要な関数呼び出し（例: `math.prod`）よりループが明快で速いことが多い。
* **定数は `Final`**: `MOD` を `Final` として明示し、読み取り専用の意図を型的に表現。
* **分岐なしの緊密ループ**: ループ内で条件分岐・関数呼び出しを避け、ブランチ予測の負担を減らす。

---

<h2 id="edgecases">エッジケースと検証</h2>

* **最小規模:** $N=2$（`routes` は 1 要素）。答えはそのまま $r_1 \bmod M$。
* **全て $1$:** $r_i=1$ のみ。答えは常に $1$。
* **大きい積:** 例えば $r_i=1000$ が続く。逐次剰余により安全に計算。
* **多数テスト:** $T$ が多い（最大 $1000$）。各ケースは $O(N)$ で独立処理可能。
* **サンプル検証:**

| 入力例（`routes`）   | 期待値（説明）               |
| --------------- | --------------------- |
| $[1,\,3]$     | $1\times3=3$        |
| $[2,\,2,\,2]$ | $2\times2\times2=8$ |

---

<h2 id="faq">FAQ</h2>

* **Q. なぜ毎回 `% 1234567` を取るのですか？**
  **A.** 途中の積が指数的に大きくなるのを避け、計算資源と時間を節約するためです。剰余環では $(ab)\bmod M \equiv ((a\bmod M)(b\bmod M))\bmod M$ が成り立ち、逐次剰余は正しさを保ちます。

* **Q. `math.prod(routes) % MOD` では駄目ですか？**
  **A.** 正しさは同じですが、中間値が巨大になり非効率（メモリ・時間）です。逐次剰余の方が安定高速です。

* **Q. `n` を使っていないのは問題では？**
  **A.** `routes` の長さが $n-1$ で与えられている前提のため、ループは `routes` を走査すれば十分です（HackerRank の入力制約に合致）。必要であれば検証用に `assert len(routes) == n-1` を付けられます（競技では通常省略）。

* **Q. 結果が $0$ になることは？**
  **A.** あり得ます。例えば $r_i$ が $M$ の倍数なら、その時点で積は $0 \bmod M$ です。これは数学的にも正当です。

---


わかりやすさ優先で、**「なぜ区間ごとの本数を掛け算するだけで総経路数になるのか」**と、**「逐次で法 $M$ を取ってよい理由」**を、図と短い式で説明します。数式は KaTeX、図は Mermaid（日本語ラベル）です。

---

## 直感から始める：独立選択の積

* 各区間 $[T_i \to T_{i+1}]$ で選べる道路は $r_i$ 通り。
* 区間は**順番に必ず全部通る**ので、区間ごとの選択は独立（同時に選ぶ組合せ）になります。
* よって総数は
  $$\text{経路総数}=\prod_{i=1}^{N-1} r_i.$$

これを **大きな数を扱わないため** に、都度 $M=1234567$ で剰余を取りながら数えます。

---

## 図1：帰納（数学的な骨格）

```mermaid
flowchart TD
  Start[命題の確認] --> Base[基底: k=0 で ans_0=1]
  Base --> Inv0["不変条件: ans_k ≡ ∏_{i=1}^{k} r_i (mod M)"]
  Inv0 --> Step["帰納: 次の区間 r_{k+1} を処理"]
  Step --> Update["更新: ans_{k+1} ← (ans_k * (r_{k+1} mod M)) mod M"]
  Update --> Reason["理由: (ab mod M) ≡ ((a mod M)(b mod M)) mod M"]
  Reason --> Inv1["結論: ans_{k+1} ≡ ∏_{i=1}^{k+1} r_i (mod M)"]
  Inv1 --> Loop["これを k=0..N-2 まで繰り返す"]
  Loop --> End["最終: ans_{N-1} ≡ ∏_{i=1}^{N-1} r_i (mod M)"]
```

**説明（対応する式）**

* **基底**：$k=0$（区間未処理）は空積なので $1$。
  $$\text{ans}_0 = 1.$$
* **帰納ステップ**：不変条件
  $$\text{ans}*k \equiv \prod*{i=1}^{k} r_i \pmod{M}$$
  を仮定し、
  $$
  \text{ans}*{k+1}
  = (\text{ans}*k \cdot (r*{k+1}\bmod M))\bmod M
  \equiv \left(\prod*{i=1}^{k} r_i\right) r_{k+1} \pmod{M}
  = \prod_{i=1}^{k+1} r_i \pmod{M}.
  $$
* **有限回の繰り返し**で $k=N-1$ に到達し、命題が確立します。

---

## 図2：逐次剰余が正しい理由（剰余環の積）

```mermaid
graph LR
  A[前段の積 a] --> B[剰余 a mod M]
  C[次の区間 b] --> D[剰余 b mod M]
  B --> E["積の剰余 ((a mod M)(b mod M)) mod M"]
  D --> E
  A --> F["直接の剰余 (ab) mod M"]
  C --> F
  E --> G[一致する性質]
  F --> G
```

**要点の式**
$$
(ab)\bmod M \equiv \bigl((a\bmod M)\cdot(b\bmod M)\bigr)\bmod M.
$$

つまり、**途中で剰余を取っても最終結果は同じ**。これが「毎ステップで `% M`」してよい数学的根拠です。

---

## 図3：具体例で見る（積の木）

例：$[r_1,r_2,r_3]=[2,3,4]$ のとき

```mermaid
flowchart TD
  S[開始 ans=1] --> X1["区間1: ans=1*2 → 2 (mod M)"]
  X1 --> X2["区間2: ans=2*3 → 6 (mod M)"]
  X2 --> X3["区間3: ans=6*4 → 24 (mod M)"]
  X3 --> R["結果 24 (mod M)"]
```

* 区間ごとに「何通り？」を掛け足すだけで、**全体の組合せ数**に到達します。

---

## 図4：不変条件の視覚化（部分積の保持）

```mermaid
graph LR
  P0[ans_0=1] --> P1["ans_1 ≡ r_1 (mod M)"]
  P1 --> P2["ans_2 ≡ r_1 r_2 (mod M)"]
  P2 --> P3["ans_3 ≡ r_1 r_2 r_3 (mod M)"]
  P3 --> PN["... 最終 ans_{N-1} ≡ ∏_{i=1}^{N-1} r_i (mod M)"]
```

* **各ステップ終了時**に、「これまでの区間の積（法 $M$）」が **必ず** `ans` に入っている、という“**不変条件**”を図で表現しています。

---

## まとめ（1枚の式）

* 経路総数（都市を番号順にすべて通る）は
  $$\prod_{i=1}^{N-1} r_i$$
* 実装では**都度剰余**で
  $$
  \text{ans}\leftarrow(\text{ans}\cdot(r_i\bmod M))\bmod M,\quad M=1234567,
  $$
  としても最終結果は**厳密に一致**。
* 有限回の更新（$N-1$ 回）で終了し、**正しさ（帰納）**と**終了性**が共に保証されます。
